In [ ]:
import torch
from transformers import AutoProcessor, AutoModelForImageTextToText
from PIL import Image
from peft import PeftModel

###############################################################################
# PART 1: MODEL INITIALIZATION
###############################################################################
def initialize_finetuned_model(base_model_path: str, finetuned_model_path: str):
    """
    Initializes the processor and integrates the fine-tuned LoRA adapter into the base model.
    """
    # Load the processor
    processor = AutoProcessor.from_pretrained(base_model_path)
    
    # Load the base model
    base_model = AutoModelForImageTextToText.from_pretrained(
        base_model_path,
        torch_dtype="auto",
        device_map="auto"
    )
    
    # Load the fine-tuned LoRA adapter
    model = PeftModel.from_pretrained(
        base_model,
        finetuned_model_path,
        torch_dtype="auto",
        device_map="auto"
    )
    model.eval()  
    return processor, model

###############################################################################
# PART 2: DATA PREPARATION (images, text, prompt)
###############################################################################
def prepare_data(buyer_image_path: str, seller_image_path: str):
    """
    Loads and resizes the images, defines buyer/seller descriptions, 
    constructs chat messages, and prepares the model inputs.
    """
    try:
        # Load & resize images
        buyer_image = Image.open(buyer_image_path).convert("RGB").resize((524, 524))
        seller_image = Image.open(seller_image_path).convert("RGB").resize((524, 524))
    except Exception as e:
        raise RuntimeError(f"Error loading images: {e}")

    # Example buyer/seller descriptions
    buyer_description = (
        """buyer reveiw
"""
    )
    
    seller_description = (
 """ seller product description
"""
    )
    
    prompt = (
       """ prompt for system"""
    )
    
    # Construct messages
    messages = [
        {
            "role": "user",
            "content": [
                {"type": "image", "image": buyer_image},
                {"type": "text", "text": buyer_description},
                {"type": "image", "image": seller_image},
                {"type": "text", "text": seller_description},
                {"type": "text", "text": prompt},
            ],
        }
    ]
    
    return buyer_image, seller_image, messages

###############################################################################
# PART 3: GENERATE AND DECODE MODEL OUTPUT
###############################################################################
def generate_analysis(processor, model, buyer_image, seller_image, messages):
    """
    Uses the loaded processor and model to transform the messages into a suitable format,
    then generates and decodes the final output.
    """
    try:
        text = processor.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
        )
        if text is None:
            raise ValueError("Generated text from apply_chat_template is None.")

        inputs = processor(
            text=[text],
            images=[buyer_image, seller_image],
            padding=True,
            return_tensors="pt",
        ).to("cuda" if torch.cuda.is_available() else "cpu")

        generated_ids = model.generate(
            **inputs,
            max_new_tokens=1500,
            temperature=0.8,
            repetition_penalty=1.2,
            top_k=50,
            top_p=0.99
        )

        output_text = processor.batch_decode(
            generated_ids,
            skip_special_tokens=True,
            clean_up_tokenization_spaces=False
        )

        return output_text[0]
    except Exception as e:
        raise RuntimeError(f"Error during processing or inference: {e}")

###############################################################################
# MAIN EXECUTION EXAMPLE
###############################################################################
if __name__ == "__main__":
    base_model_path = "qwen_model"  # Base model directory
    finetuned_model_path = "finetuned"  # Fine-tuned model directory

    processor, model = initialize_finetuned_model(base_model_path, finetuned_model_path)

    buyer_image_path = "image/path.jpg"
    seller_image_path = "image/path.jpg"
    buyer_image, seller_image, messages = prepare_data(buyer_image_path, seller_image_path)

    # 3) Generate and print analysis
    analysis_text = generate_analysis(processor, model, buyer_image, seller_image, messages)
    print("Generated Structured Analysis:\n", analysis_text)

